### Código a partir do Vídeo:

https://www.youtube.com/watch?v=pDA4mncyJ8Q&ab_channel=AlexandreH

https://www.youtube.com/watch?v=3cwkEOUscXA&ab_channel=AlexandreH

In [28]:
import cv2

#pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
import pytesseract

import pandas as pd



In [2]:
def encontrar_a_placa (imagem_carro):
    img = cv2.imread(imagem_carro)
    #img = cv2.imread('../Imagem/CARRO4.JPG') 
    #cv2.imshow('img', img)

    # Escalas de Cinza para o openCV indentificar
    cinza = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #cv2.imshow('cinza', cinza)

    # Converter em preto e branco, para binarizar a imagem com as escalas de cores demonstradas com os números 
    # indo até 128 no coloração preta e 255 na coloração branca

    _, bin = cv2.threshold (cinza, 60, 220, cv2.THRESH_BINARY) 

    # TRESH_BINARY traz o limite para as cores 
    # _, essa variável necessita de um retorno

    #cv2.imshow('bin', bin)

    # Desfocar a imagem para identificar melhor as formas geométricas, encontrando melhor os contornos

    desfocar = cv2.GaussianBlur(bin, (5,5), 0)
    cv2.imshow('Imagem_desfocada', desfocar)

    # Agora procurar os contornos

    contornos, hierarquia = cv2.findContours(desfocar, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    #cv2.drawContours(img, contornos, -1,  (0 , 255, 0), 2 )

    # o -1 é o ID dos contornos, como não precisa dessa informação colocamos o -1
    # 0 255 0 ---> Cor do contorno, no caso verde 
    # 2 ------> largura da borda do contorno

    #cv2.imshow('Contornos', img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

    # fazer contornos quadrados 

    for c in contornos:
    
        perimetro = cv2.arcLength (c, True)
        aprox = cv2.approxPolyDP(c, 0.03*perimetro, True)
        
        if perimetro > 500:
            aprox = cv2.approxPolyDP(c, 0.03*perimetro, True)
            
            # Igualar a quatro cantos e aproximar a um retângulo

            if len (aprox) == 4:
                (x, y, altura, largura) = cv2.boundingRect(c)
                cv2.rectangle (img, (x,y), (x+altura, y+largura), (0,255,0), 2)

                temporario = img[y:y + largura, x:x + altura]
                cv2.imwrite('../Imagem/Imagem_temporaria.jpg', temporario)

    cv2.imshow ('Placa_contornada', img)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()



In [3]:
def pre_processamento():
    
    imagem_placa = cv2.imread('..\Imagem\Imagem_temporaria.jpg') 

    if imagem_placa is None:
        return
    
    # Aumentar a imagem para um melhor ajuste no tesseract

    img_aumentada = cv2.resize(imagem_placa, None, fx=4, fy=4, interpolation=cv2.INTER_CUBIC)
    
    # O 50 e o 255 são colocados na placa, é a margem do quanto de branco e cinza serão adicionados na imagem para realçar as letras 

    img_cinza = cv2.cvtColor(img_aumentada, cv2.COLOR_BGR2GRAY)
    
    #img_cinza = cv2.cvtColor(img_aumentada, cv2.COLOR_BGR2GRAY)

    _, img_binaria = cv2.threshold (img_cinza, 65, 200, cv2.THRESH_BINARY)

    # Salvar a imagem binarizada e em cinza

    cv2.imwrite('../Imagem/placa_binarizada.jpg', img_binaria)
    cv2.imwrite('../Imagem/placa_binarizada_png.png', img_binaria)

    #cv2.imshow("ler_placa", imagem_placa)
    #cv2.imshow("placa_cinza", img_cinza)
    cv2.imshow("placa_binaria", img_binaria)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()


In [4]:
def Ocr_da_Placa():
    
    img_ocr = cv2.imread('../Imagem/placa_binarizada.jpg')

    config = r'-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWYXZ0123456789 --psm 6'
    # A saida será em inglês para ler melhor a placa em lang = eng

    saida = pytesseract.image_to_string(img_ocr, lang="eng", config=config)
    print(saida)

    

In [ ]:
def tirar_foto:

    from picamera import PiCamera
    from time import sleep

    camera = PiCamera()

    camera.resolution = (600,450)

    camera.start_preview()
    for i in range(5):
        sleep(2)
        camera.capture('..Imagem/Foto%s.jpg' % i)
    camera.stop_preview() 

In [8]:
if __name__ == "__main__":

    tirar_foto()

    for i in range (5):
        
        imagem_carro = '../Imagem/Foto%s.jpg' % i;

        encontrar_a_placa(imagem_carro)

        pre_processamento()

        Ocr_da_Placa()


SHS311



In [75]:
Banco_de_dados = pd.read_csv('..\Biblioteca\Moradores.csv')


In [78]:
Banco_de_dados.head(10)

,PLACA,MORADOR,NUMERO DA RESIDENCIA
0,ATF7354,JOAO,1
1,BEE4R22,PAULO,2
2,ATF7355,MARIA,3
3,BEE4R23,SABRINA,4
4,SHS311,JORGE,5
5,BEE4R24,MARCELO,6
6,ATF7357,LUIS,7
7,BEE4R25,GUSTAVO,8
8,ATF7358,THIAGO,9
9,1BEE4R26,ARTUR,10


In [65]:
Banco_de_dados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, ATF7354 to BEE4R26
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PLACA                 10 non-null     object 
 1   MORADOR               10 non-null     int64  
 2   NUMERO DA RESIDENCIA  0 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 320.0+ bytes


In [77]:
Banco_de_dados['PLACA'] == 'ATF7354'

0     True
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
Name: PLACA, dtype: bool